Europos atvirų duomenų portalas atsiuntė klausimyną, kurio pagalba vertins
atvirų duomenų progresą Lietuvoje. Aš gavau užduotį paanalizuoti [Lietuvos
atvirų duomenų katalogo](https://data.gov.lt/) (ADK) duomenis ir atsakyti į kai
kuriuos klausimyno klausimus.

ADK viešojo API dokumentacija pasiekiama šiuo adresu:

https://data.gov.lt/public/api/1

Dokumentacijoje nurodyta, kad ADK įgyvendina tik vieną API prieigos tašką:

https://data.gov.lt/public/api/1/action/package_search

Kuris kaip suprantu atitinka CKAN
[package_search](https://docs.ckan.org/en/2.9/api/#ckan.logic.action.get.package_search),
o pats CKAN po dangčiu naudoja
[SOLR](https://lucene.apache.org/solr/guide/6_6/common-query-parameters.html).


# Duomenų surinkimas (1 bandymas)

Bandau žiūrėti ką grąžina API prieigos taškas:


In [41]:
!http -b 'https://data.gov.lt/public/api/1/action/package_search?rows=1'

{
    "help": "https://data.gov.lt/public/api/1/action/help_show?name=package_search",
    "result": {
        "count": 1,
        "results": [
            {
                "author": "",
                "authorEmail": "",
                "creatorUserId": "",
                "id": "{24C009EF-BD65-1890-F505-D0FBB3C16F43}",
                "is_open": false,
                "metadata_created": "2020-06-10T11:16:37",
                "metadata_modified": "2020-08-05T13:00:54",
                "notes": "Lietuvos nacionalinio atlaso žemėlapis - „Paros vidutinė oro temperatūra pakyla aukščiau ir nukrinta žemiau 0°C“. Autorius:A. Bukantis. Kartografas: G. Reginienė. Žemėlapis priklauso Nacionalinio atlaso skyriui - \"Gamta ir kraštovaizdis\", temai - \"Klimatas\"",
                "numResources": 0,
                "numTags": 0,
                "open": false,
                "private": false,
                "state": "active",
                "title": "WMS. Lietuvos nacionalinio atlaso žemėlapi

Kaip suprantu, man reikia susirinkti visus puslapius ir sujungti juos į vieną
didelį sąrašą. Kadangi portale yra mažiau nei 1000 duomenų rinkinių, tai visus
rinkinius užsikrauti į RAM neturėtų būti problema.

In [44]:
import requests

start: int = 0
rows: int = 50
datasets = []
while True:
    url = 'https://data.gov.lt/public/api/1/action/package_search'
    resp = requests.get(url, params={
        'rows': rows,
        'start': start,
        'sort': 'metadata_created',
    })
    assert resp.ok, resp.url
    results = resp.json()['result']['results']
    if results:
        datasets += results
        start += rows
    else:
        break

AssertionError: https://data.gov.lt/public/api/1/action/package_search?rows=50&start=250&sort=metadata_created

O, pavyko gauti 500 klaidą, ir jokios informacijos apie tai, kas galėjo būti
blogai.

Gal reikia pabandyti pasiimti visus duomenis vienos užklausos pagalba?

In [42]:
resp = requests.get('https://data.gov.lt/public/api/1/action/package_search', {
    'rows': 1000,
})
assert resp.ok

AssertionError: 

Neveikia. Įdomu, kaip duomenis
[susiimportavo](https://www.europeandataportal.eu/data/datasets?catalog=data-gov-lt)
Europos atvirų duomenų portalas?